**API 정상 작동 되는지 확인**

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
#API 키 추적
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

[API KEY]
sk-it3BehgAOcD6sOyc_B7H8KJOWjv_ihVEtPLdJNRx-oT3BlbkFJ921npAAPi54rP5Ei96bZ1EarSEMUeinhWaUpbfwbgA


**OpenAI 정상 작동 되는지 확인**

In [2]:
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o",  # 모델명
    #max_tokens = 사용할 토큰 수
)

# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
print(f"[답변]: {llm.invoke(question)}")

[답변]: content='대한민국의 수도는 서울입니다.' response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 24, 'total_tokens': 39}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-3bffe716-3d8c-49c7-aaea-052a5238972f-0' usage_metadata={'input_tokens': 24, 'output_tokens': 15, 'total_tokens': 39}


**템플릿 사용하기**


**template: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 {}는 변수를 나타냅니다.**


**input_variables: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.**

In [3]:
from langchain_core.prompts import PromptTemplate
# template 정의
template = "{country}의 수도는 어디인가요?"
# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [4]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [5]:
from langchain_openai import ChatOpenAI

# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o",  # 모델명
    max_tokens = 15
)


chain = prompt | model

data = {"topic": "인공지능 모델의 학습 원리"}

ouput = chain.invoke(data)

print(ouput)

content='물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명' response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 22, 'total_tokens': 37}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'length', 'logprobs': None} id='run-90e3a6f7-d6fe-49d4-8799-b182326d7f2f-0' usage_metadata={'input_tokens': 22, 'output_tokens': 15, 'total_tokens': 37}


**출력 파서까지 체인 연결**

In [ ]:
from langchain_core.output_parsers import StrOutputParser


prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o",  # 모델명
)

data = {"topic": "인공지능 모델의 학습 원리"}

output_parser = StrOutputParser()

chain = prompt | model | output_parser


output = chain.invoke(data)


**LLM 실시간 출력기능 stream**

In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델을 인스턴스화합니다.
model = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o",  # 모델명
    max_tokens = 50
)
# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대하여 2문장으로 간단하게 설명해줘.")
# 프롬프트와 모델을 연결하여 대화 체인을 생성합니다.
chain = prompt | model | StrOutputParser()

for token in chain.stream({"topic": "멀티모달"}):
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

멀티모달은 텍스트, 이미지, 오디오 등 여러 형태의 데이터를 동시에 처리하고 이해하는 기술을 의미합니다. 이를 통해 다양한 데이터 소스를 결합하여 더 풍부하고 정확한 정보를 제공할 수 있습니다.

**만약 바로 한 번에 답변을 받고 싶으면 -> invoke**

**여러 개의 invoke를 실행시키고 싶으면 -> batch**

---

**a 를 붙히면 비동기적으로 실행시킬 수도 있다.**

**두 개의 모델을 동시에 사용하고 싶을 때 - RunnableParallel -**

In [14]:
from langchain_core.runnables import RunnableParallel

model = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o",  # 모델명
    max_tokens = 30
)

# {country} 의 수도를 물어보는 체인을 생성합니다.
chain1 = (
    PromptTemplate.from_template("{country} 의 수도는 어디야?")
    | model
    | StrOutputParser()
)

# {country} 의 면적을 물어보는 체인을 생성합니다.
chain2 = (
    PromptTemplate.from_template("{country} 의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성합니다.
combined = RunnableParallel(capital=chain1, area=chain2)

output = combined.invoke({"country": "대한민국"})



In [16]:
output

{'capital': '대한민국의 수도는 서울입니다. 서울은 대한민국의 정치, 경제, 문화의 중심지로서 많은 중요한 기관과 기업들이 위치해',
 'area': '대한민국의 면적은 약 100,210 평방킬로미터(약 38,691 평방마일)입니다.'}

**chain 을 invoke() 하여 실행할 때는 입력 데이터의 타입이 딕셔너리여야 한다.**

In [18]:
from langchain_core.runnables import RunnablePassthrough

# runnable
x = RunnablePassthrough().invoke({"num": 10})

type(x)

dict

**RunnablePassthrough를 사용하면 원하는 딕셔너리 데이터를 추가 생성할 수 있다.**

In [19]:
# 입력 키: num, 할당(assign) 키: new_num
x = (RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

x

{'num': 1, 'new_num': 3}

**RunnableParallel을 사용하면 나만의 RunnablePassthrough를 만들 수도 있음.**

In [20]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스를 생성합니다. 이 인스턴스는 여러 Runnable 인스턴스를 병렬로 실행할 수 있습니다.
runnable = RunnableParallel(
    # RunnablePassthrough 인스턴스를 'passed' 키워드 인자로 전달합니다. 이는 입력된 데이터를 그대로 통과시키는 역할을 합니다.
    passed=RunnablePassthrough(),
    # 'extra' 키워드 인자로 RunnablePassthrough.assign을 사용하여, 'mult' 람다 함수를 할당합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값을 3배로 증가시킵니다.
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 'modified' 키워드 인자로 람다 함수를 전달합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값에 1을 더합니다.
    modified=lambda x: x["num"] + 1,
)

# runnable 인스턴스에 {'num': 1} 딕셔너리를 입력으로 전달하여 invoke 메소드를 호출합니다.
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

**Chain에 적용한 예시**

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI(
    temperature=0.1,  
    model_name="gpt-4o",
    max_tokens=30
)

output_parser = StrOutputParser()

chain = {"topic" : RunnablePassthrough()} | prompt | model | output_parser


output = chain.invoke("사과")

output

'사과는 많은 사람들이 좋아하는 과일 중 하나입니다. 사과는 나무에서 자라며, 다양한 색상과 크기로 나'

**RunnableLambda를 사용하면 함수 아웃풋을 Runnable 하게 만들어서 사용할 수 있음**

In [34]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from datetime import datetime

def get_today(input=None):
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")

prompt = PromptTemplate.from_template(
    "{today} 가 생일인 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)

model = ChatOpenAI(
    temperature=0.1,  
    model_name="gpt-3.5-turbo",
    max_tokens=30
)

output_parser = StrOutputParser()

chain = ({"today" : RunnableLambda(get_today) , "n" : RunnablePassthrough()} 
         | prompt 
         | model 
         | output_parser)


output = chain.invoke(1)

output

'유명인: 박보검 (Park Bo Gum)\n생년월일: 1993년 6'

**itemgetter 를 사용하면 RunnableLambda에 인자를 넣을 수 있슴**

In [36]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from datetime import datetime
from operator import itemgetter

def text_length(x):
    # 오늘 날짜를 가져오기
    return len(x)

prompt = PromptTemplate.from_template(
    "{a} + {b} 는 얼마야?"
)

model = ChatOpenAI(
    temperature=0.1,  
    model_name="gpt-3.5-turbo",
    max_tokens=50
)

output_parser = StrOutputParser()

chain = ({"a" : itemgetter("word1")| RunnableLambda(text_length) , "b" : itemgetter("word2") | RunnableLambda(text_length)} 
         | prompt 
         | model 
         | output_parser)


output = chain.invoke({"word1" : "안녕하세요." , "word2" : "하이"})

output

'6 + 2 = 8'